In [1]:
# Custom extra width cells
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%matplotlib inline
import requests,pandas as pd

In [3]:
data_dir = "../data/"

api_key = ""
# Get the key
# with open(f'census_key.txt') as key:
#     api_key=key.read().strip()
    
# # print(api_key)

FileNotFoundError: [Errno 2] No such file or directory: 'census_key.txt'

In [ ]:
def get_nan_col_stats(df_missing):
    # Get the missing values per each column 
    missing_series = df_missing.isna().sum()

    # Concatunate each column's type so we have a better idea for imputation
    missing_types_df = pd.concat([missing_series.rename('Missing'), df_missing.dtypes.rename('Dtype')], axis=1)
    missing_types_df.sort_values(by=['Missing'], ascending=False, inplace=True)
    display(missing_types_df)

### GEO_ID Mapping via Relationship file
Since the postal code average income api response only returns a GEO_ID per zipcode, we first want to get all the GEO_IDs and their corresponding state and counties to cross reference in the next step.

In [ ]:
# All relationdship files: https://www2.census.gov/geo/pdfs/maps-data/data/rel/
# https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_place_rel_10.txt

# https://opendata.stackexchange.com/a/13731
# Read in the ZCTA5 to PLACE & STATE file
zcta_place_relations_df = pd.read_csv("zcta_place_rel_10.txt")

# Only keep the columns we want to merge
zcta_place_relations_df = zcta_place_relations_df[["ZCTA5", "STATE", "PLACE"]]

# Need to rename the columns so they can be matched on the next step
new_col_names = {"STATE": "State_id", "PLACE":"Place_id"}
zcta_place_relations_df.rename(columns = new_col_names, inplace = True)

# Output stats
print(f"Place list amount -> {len(zcta_place_relations_df.index)}")
display(zcta_place_relations_df.head())

# Test a match 28277
zcta_place_relations_df.loc[zcta_place_relations_df.index == 28277]

### Matching Place and State IDs to their actual names
We first want to request the actual names for the state and place ids. And then we can add the ZCTA to this file, which is needed for matching the houeshold incomes.

In [ ]:
# Needed for census GEO_IDs call
year='2018'
dsource='acs' # American Cummunity Survey
dname='acs5' # 5 year survey

# B19001_001 -> HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)
cols='NAME,GEO_ID'
geography_state = 'place:*'
# https://api.census.gov/data/2018/acs/acs5?key=&get=B19001_001E,NAME,COUNTY,GEO_ID&for=zip%20code%20tabulation%20area:*%20state:*


def map_geoids_to_df(response_geos):
    # Read the data to json
    data = response_geos.json()

    # Create the df
    df_geoids = pd.DataFrame(data[1:], columns=data[0]).\
        rename(columns={"NAME": "Place_state", "place": "Place_id",
                        "state": "State_id", "GEO_ID":"gid"})


    # https://stackoverflow.com/a/61313413/8970591
    # Seperate the format 'place, state' to 2 seperate columns
    df_geoids[['Place', 'State']] = df_geoids['Place_state'].str.split(',', n=1, expand=True)
    # df_geoids.drop(columns=['County_state', 'State_id', 'County_id'], inplace=True)
#     df_geoids.set_index('gid', inplace=True)

    # Convert to integer so we can match it with the place/state file indexes
    df_geoids['Place_id'] = df_geoids['Place_id'].astype(int)
    df_geoids['State_id'] = df_geoids['State_id'].astype(int)


    # Ouput the results
    display(df_geoids.head())
    
    return df_geoids

def geo_id_mapping_df():

    base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
    # url_zip = "https://api.census.gov/data/2017/acs/acs5?get=NAME,group(B19013)&for={geography}"


    # &for=zip%20code%20tabulation%20area:*"
    data_url = f'{base_url}?key={api_key}&get={cols}&for={geography_state}'
    
    # Get the census data
    response_geos = requests.get(data_url)
       
    # Convert to df and return
    return map_geoids_to_df(response_geos)


In [ ]:
# Now we want to match the actual place and state names to this file
geo_ids_df = geo_id_mapping_df()
display(geo_ids_df.info())

# Finally merge the the ZCTA to this file
# Merge place and state id with each ZCTA row
zcta_place_best = geo_ids_df.merge(zcta_place_relations_df, how = 'inner', on = ['State_id', 'Place_id'])

# Now we can remove the columns we do not need anymore
drop_cols = ['gid', 'Place_id', 'State_id']
zcta_place_best.drop(columns = drop_cols, inplace=True)

# Set to index so we can match it
zcta_place_best.set_index('ZCTA5', inplace=True)
zcta_place_best.index.name = 'ZCTA5'

# Ouput the results
display(zcta_place_best)



In [ ]:
# Lastly do our nan check to see if anything did not match
get_nan_col_stats(zcta_place_best)


### Getting coordinates for each zipcode
We want this data, so we can add these columns to our income df

In [ ]:
# http://download.geonames.org/export/zip/
dtypes_dict = {
    0: str, # country code  
    1: str,  # postal code 
    2: str,  # place name 
    
    3: str,  # admin name1  order subdivision (state) varchar(100)
    4: str, # admin code  subdivision (state) varchar(20)
    5: str, # admin name2  subdivision (county/province) varchar(100)
    6: str, # admin code  (county/province) varchar(20)
    7: str, # admin name3 (community) varchar(100)
    8: str, # admin code3  (community) varchar(20)
    
    9: float, # latitude
    10: float, # longitude
    11: float, # accuracy of lat/lng
}

us_zip_df = pd.read_csv(f"{data_dir}/US_zip.txt", sep="\t", 
            usecols=(0, 1, 2, 3, 4, 5, 6,  9, 10, 11), index_col=0,
            names=('country_code', 'postal_code', 'place_name', 
                   'state', 'state_code', 'county_sub', 'county_code',
                   'latitude', 'longitude', 'acc_lat_long'), dtype=dtypes_dict)


# We only want to keep the coordinates and their accuracy
us_zip_df = us_zip_df[["postal_code", "latitude", "longitude", "acc_lat_long"]]
us_zip_df['zipcode_ta'] = us_zip_df['postal_code'].astype(int)

# Drop any nan values for the selection of columns we need
us_zip_df.dropna(inplace=True) 

# # Set index to zipcode for merging
us_zip_df.set_index('zipcode_ta', inplace=True)

# Validate the output
display(us_zip_df.head())

get_nan_col_stats(us_zip_df)

### Getting the average household income data and matching coordinates
We will now fetch the average household income data by zipcode and then match the county and city name

In [ ]:
# American Cummunity Survey
dsource='acs'
dname='acs5' # 5 year survey

# B19001_001 -> HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)
cols='B19001_001E'

# https://api.census.gov/data/2018/acs/acs5?key=&get=B19001_001E,NAME,COUNTY,GEO_ID&for=zip%20code%20tabulation%20area:*%20state:*
geography_zip = 'zip%20code%20tabulation%20area:*'


def add_coordinates_to_zip(df_inc):
    # Add coordinates to the zipcodes by merging
    # Left join since we want to add the columns for the df_inc table
    df_final = df_inc.merge(us_zip_df, left_index=True, right_index=True, how='left')
    
    # Drop unmatched columns
    df_final.dropna(inplace=True)

    # Ouput results
    # print(len(df_inc.index))
    # print(len(us_zip_df.index))
    # print(len(df_final.index))
    
    return df_final
    

def create_and_income_df(response_inc, year):
    # Read the data to json
    data = response_inc.json()

    # Create the df
    df_inc = pd.DataFrame(data[1:], columns=data[0]).\
        rename(columns={"B19001_001E": "Avg_household_inc",
                        "zip code tabulation area": "zipcode_ta"})


    # Convert to integer so we can match it with the place/state file indexes
    df_inc['zipcode_ta'] = df_inc['zipcode_ta'].astype(int)
    df_inc.set_index('zipcode_ta', inplace=True)
    
    # Drop columns with no household inc info
    df_inc.dropna(inplace=True)

    # Add coordinates
    df_inc = add_coordinates_to_zip(df_inc)
    
    # Ouput the results
    # display(df_inc.head())
    
    return df_inc


def get_acs5_inc_cencus_data_for_year_as_df(year):
    # Get base url for year
    base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'

    # Get final url and make the request
    data_url = f'{base_url}?get={cols}&for={geography_zip}&key={api_key}'
    response = requests.get(data_url)
    
    return create_and_income_df(response, year)



In [ ]:
# Each year we want the data from
# 2019 and 2020 are not available yet...
years = ['2015', '2016', '2017', '2018']

for year in years:
    print(f"Requesting income data for year {year}....")

    # Get this year's df with ZCTA as index
    df_inc = get_acs5_inc_cencus_data_for_year_as_df(year)
    
    # Merge place and state with each ZCTA row
    df_final = pd.merge(df_inc, zcta_place_best, left_index=True, right_index=True)
    df_final.index.name = 'ZCTA5'
    
    # Drop columns we don't need anymore
    
    # Output the results
    # display(df_final.head())
    
    # Validate of each ZCTA is matched
    get_nan_col_stats(df_final)
    
    # Write to disk
    print(f"Going to write income data for year {year}....")
    df_final.to_csv(f"{data_dir}/census_inc_{year}", encoding='utf-8')

    
print("Done!")
